In [20]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
import geoviews as gv
import pyproj
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key


In [21]:
crime_data_to_load = "2022-09-west-midlands-street.csv"

crime_data = pd.read_csv(crime_data_to_load)

crime_data.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,NaN,2022-09,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Birmingham,Anti-social behaviour,NaN,NaN
1,eeef87491d8fda65a91499ba406e040cdb2486bb2846c5...,2022-09,West Midlands Police,West Midlands Police,-1.850772,52.588709,On or near Seymour Gardens,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified,NaN
2,9d420a3e1fc51ed2cb4751f9b1c5f5d7788d4ecd68e6b3...,2022-09,West Midlands Police,West Midlands Police,-1.847899,52.591564,On or near Royal Park Close,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified,NaN
3,8cbf9e0b0caea93c62d392f4000c8196239386b2db1ad0...,2022-09,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified,NaN
4,5b23b5d9a702b22e91c949c576a5d91b7522c30c3e96dc...,2022-09,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified,NaN


In [22]:
crime_data.isna().sum()


Crime ID                  1496
Month                        0
Reported by                  0
Falls within                 0
Longitude                    0
Latitude                     0
Location                     0
LSOA code                    0
LSOA name                    0
Crime type                   0
Last outcome category     1496
Context                  31697
dtype: int64

In [23]:
crime_data.drop("Context",inplace=True,axis=1)
crime_data


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category
0,NaN,2022-09,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Birmingham,Anti-social behaviour,NaN
1,eeef87491d8fda65a91499ba406e040cdb2486bb2846c5...,2022-09,West Midlands Police,West Midlands Police,-1.850772,52.588709,On or near Seymour Gardens,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified
2,9d420a3e1fc51ed2cb4751f9b1c5f5d7788d4ecd68e6b3...,2022-09,West Midlands Police,West Midlands Police,-1.847899,52.591564,On or near Royal Park Close,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified
3,8cbf9e0b0caea93c62d392f4000c8196239386b2db1ad0...,2022-09,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified
4,5b23b5d9a702b22e91c949c576a5d91b7522c30c3e96dc...,2022-09,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified
...,...,...,...,...,...,...,...,...,...,...,...
31692,b3fca1c631f57f1816961c5c89afadd9cd24025eac669b...,2022-09,West Midlands Police,West Midlands Police,-2.120997,52.579436,On or near Dartmouth Street,E01010530,Wolverhampton,Violence and sexual offences,Under investigation
31693,f6846073eefdfdafa7de407d53ad093701bac32cffdfc8...,2022-09,West Midlands Police,West Midlands Police,-2.117684,52.582451,On or near Duke Street,E01010530,Wolverhampton,Violence and sexual offences,Unable to prosecute suspect
31694,0caa10f10bfcdc1fd74f26547be0963cf4d44c3694b21f...,2022-09,West Midlands Police,West Midlands Police,-2.120010,52.585361,On or near Union Street,E01010530,Wolverhampton,Violence and sexual offences,Investigation complete; no suspect identified
31695,a4c7cf185a1330eca90053ba9ff78c869943cae08f95ac...,2022-09,West Midlands Police,West Midlands Police,-2.120010,52.585361,On or near Union Street,E01010530,Wolverhampton,Violence and sexual offences,Under investigation


In [24]:
crime_data.dropna(how="any", inplace=True)
crime_data


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category
1,eeef87491d8fda65a91499ba406e040cdb2486bb2846c5...,2022-09,West Midlands Police,West Midlands Police,-1.850772,52.588709,On or near Seymour Gardens,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified
2,9d420a3e1fc51ed2cb4751f9b1c5f5d7788d4ecd68e6b3...,2022-09,West Midlands Police,West Midlands Police,-1.847899,52.591564,On or near Royal Park Close,E01009417,Birmingham,Burglary,Investigation complete; no suspect identified
3,8cbf9e0b0caea93c62d392f4000c8196239386b2db1ad0...,2022-09,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified
4,5b23b5d9a702b22e91c949c576a5d91b7522c30c3e96dc...,2022-09,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified
5,c80a24d33359d13410b2299d303ec184e5b8880ac16b37...,2022-09,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Birmingham,Vehicle crime,Investigation complete; no suspect identified
...,...,...,...,...,...,...,...,...,...,...,...
31692,b3fca1c631f57f1816961c5c89afadd9cd24025eac669b...,2022-09,West Midlands Police,West Midlands Police,-2.120997,52.579436,On or near Dartmouth Street,E01010530,Wolverhampton,Violence and sexual offences,Under investigation
31693,f6846073eefdfdafa7de407d53ad093701bac32cffdfc8...,2022-09,West Midlands Police,West Midlands Police,-2.117684,52.582451,On or near Duke Street,E01010530,Wolverhampton,Violence and sexual offences,Unable to prosecute suspect
31694,0caa10f10bfcdc1fd74f26547be0963cf4d44c3694b21f...,2022-09,West Midlands Police,West Midlands Police,-2.120010,52.585361,On or near Union Street,E01010530,Wolverhampton,Violence and sexual offences,Investigation complete; no suspect identified
31695,a4c7cf185a1330eca90053ba9ff78c869943cae08f95ac...,2022-09,West Midlands Police,West Midlands Police,-2.120010,52.585361,On or near Union Street,E01010530,Wolverhampton,Violence and sexual offences,Under investigation


In [25]:
crime_data = crime_data.rename(columns= {
                                                "LSOA name":"City Name"})
new_crime_data = crime_data[["City Name", "Month", "Crime type", "Reported by", "Falls within", "Longitude", "Latitude", "Location", "LSOA code","Last outcome category", "Crime ID"]]
#new_crime_data = new_crime_data.set_index("City Name")
new_crime_data.head()


,City Name,Month,Crime type,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,Last outcome category,Crime ID
1,Birmingham,2022-09,Burglary,West Midlands Police,West Midlands Police,-1.850772,52.588709,On or near Seymour Gardens,E01009417,Investigation complete; no suspect identified,eeef87491d8fda65a91499ba406e040cdb2486bb2846c5...
2,Birmingham,2022-09,Burglary,West Midlands Police,West Midlands Police,-1.847899,52.591564,On or near Royal Park Close,E01009417,Investigation complete; no suspect identified,9d420a3e1fc51ed2cb4751f9b1c5f5d7788d4ecd68e6b3...
3,Birmingham,2022-09,Vehicle crime,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Investigation complete; no suspect identified,8cbf9e0b0caea93c62d392f4000c8196239386b2db1ad0...
4,Birmingham,2022-09,Vehicle crime,West Midlands Police,West Midlands Police,-1.849790,52.590937,On or near Walsall Road,E01009417,Investigation complete; no suspect identified,5b23b5d9a702b22e91c949c576a5d91b7522c30c3e96dc...
5,Birmingham,2022-09,Vehicle crime,West Midlands Police,West Midlands Police,-1.848972,52.588428,On or near Woodside,E01009417,Investigation complete; no suspect identified,c80a24d33359d13410b2299d303ec184e5b8880ac16b37...


In [26]:

new_crime_data_bham = new_crime_data.loc[(new_crime_data["City Name"]=="Birmingham")]
new_crime_data_cov = new_crime_data.loc[(new_crime_data["City Name"]=="Coventry")]
new_crime_data_dud = new_crime_data.loc[(new_crime_data["City Name"]=="Dudley")]
new_crime_data_wham = new_crime_data.loc[(new_crime_data["City Name"]=="Wolverhampton")]

In [27]:
crime_plot_bham = new_crime_data_bham.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "Crime type"
)

# Display the map plot
crime_plot_bham

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Crime type)

In [28]:
crime_plot_cov = new_crime_data_cov.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "Crime type"
)

# Display the map plot
crime_plot_cov

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Crime type)

In [29]:
crime_plot_dud = new_crime_data_dud.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "Crime type"
)

# Display the map plot
crime_plot_dud

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Crime type)

In [44]:
crime_plot_wham = new_crime_data_wham.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "Crime type"
)

# Display the map plot
crime_plot_wham

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Crime type)